In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import numpy as np

import matplotlib as mpl

import IPython.display as display
import PIL.Image
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image

In [ ]:
# url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'

In [ ]:
# Download an image and read it into a NumPy array.
def download(url, max_dim=None):
    name = url.split('/')[-1]
    image_path = tf.keras.utils.get_file(name, origin=url)
    img = PIL.Image.open(image_path)
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    return np.array(img)

def get_image(image_path, max_dim=None):
    img = PIL.Image.open(image_path)
    img = img.convert('RGB')
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    return np.array(img)

# Normalize an image
def deprocess(img):
    img = 255*(img + 1.0)/2.0
    return tf.cast(img, tf.uint8)

# Display an image
def show(img):
    display.display(PIL.Image.fromarray(np.array(img)))


def show_save_img(img, epoch):
    display.clear_output(wait=True)
    p_img = PIL.Image.fromarray(np.array(img))
    display.display(p_img)
    p_img.save('./output/image_{:04d}.png'.format(epoch))    

    
original_img = get_image('./source10.png')
# original_img = download(url)
print(original_img.shape)
show(original_img)

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [ ]:
# print(base_model.summary())

In [ ]:
# Maximize the activations of these layers
names = ['mixed5', 'mixed6']
names = ['mixed3', 'mixed8']
# names = ['mixed3']
layers = [base_model.get_layer(name).output for name in names]

# Create the feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

In [ ]:
# dream_model.summary()

In [ ]:
def calc_loss(img, model):
    # Pass forward the image through the model to retrieve the activations.
    # Converts the image into a batch of size 1.
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)

    losses = []
    for act in layer_activations:
        loss = tf.math.reduce_mean(act)
        losses.append(loss)

    return  tf.reduce_sum(losses)

In [ ]:
class DeepDream(tf.Module):
    def __init__(self, model):
        self.model = model
        self.total_variation_weight = 0

    @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.float32),)
    )
    def __call__(self, img, steps, step_size):
        print("Tracing")
        loss = tf.constant(0.0)
        for n in tf.range(steps):
            with tf.GradientTape() as tape:
                # This needs gradients relative to `img`
                # `GradientTape` only watches `tf.Variable`s by default
                tape.watch(img)
                loss = calc_loss(img, self.model)
                loss += self.total_variation_weight*tf.image.total_variation(img)

            # Calculate the gradient of the loss with respect to the pixels of the input image.
            gradients = tape.gradient(loss, img)

            # Normalize the gradients.
            gradients /= tf.math.reduce_std(gradients) + 1e-8 

            # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
            # You can update the image by directly adding the gradients (because they're the same shape!)
            img = img + gradients*step_size
            img = tf.clip_by_value(img, -1, 1)

        return loss, img

In [ ]:
deepdream = DeepDream(dream_model)

In [ ]:
def run_deep_dream_simple(img, begin = 0, steps=100, batch_size=100, alpha=0.01):
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    img = tf.convert_to_tensor(img)
    alpha = tf.convert_to_tensor(alpha)

    for epoch in range(begin, begin + steps):
        loss, img = deepdream(img, batch_size, tf.constant(alpha))

        if epoch % 2 == 0:
            show_save_img(deprocess(img), epoch)
            print ("Step {}, loss {}".format(epoch, loss))

    show_save_img(deprocess(img), epoch)
    return result

In [ ]:
result = run_deep_dream_simple(img=original_img, begin = 0, steps=100, alpha=0.01)